## Album Data Collection

In [2]:
# import libraries
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import csv
import re
from tqdm import tqdm

In [3]:
# connect to Spotify Client
cid ='d7fed0d1ecac4b5184554706d76d5a22'
secret ='0c1758d9564b4659b730f2a05c104ccb'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
#insert the URI as a string into the function
def add_track(df, uri_info):
    one = sp.track(uri_info, market='US')  
    cover = None
    if len(one['album']['images']) > 0:
        cover = one['album']['images'][0]['url']
    return pd.concat([df, pd.DataFrame({
    'uri':[one['uri']],
    'track':[one['name']],
    'artist': [one['artists'][0]['name'].lower()],
    'cover image': [cover]})], ignore_index=True)
    
#     return df2, artist[0]

In [5]:
df = pd.DataFrame()
df = add_track(df, 'spotify:track:2ZWlPOoWh0626oTaHrnl2a')
df = add_track(df, 'spotify:track:2ZWlPOoWh0626oTaHrnl2a')
df.loc[0]['cover image']

'https://i.scdn.co/image/ab67616d0000b273c5649add07ed3720be9d5526'

In [6]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(re.sub(r'[^A-Za-z0-9 ]+', '', artistname).replace(' ','-'))
    songname2 = str(re.sub(r'[^A-Za-z0-9 ]+', '', songname).replace(' ','-'))
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
#     print('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", id_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-6 jYfhrf")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, i):
    test = scrape_lyrics(df1.loc[i]['artist'], df1.loc[i]['track'])
    df1.loc[i, 'lyrics'] = test
    return df1


In [7]:
# read in all tracks

df = pd.DataFrame()
df = pd.read_pickle("all_data24.pkl")
        
with open('genres_v2.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in tqdm(csv_reader, total=17000):
        # if line_count < 15525:
        #     line_count += 1
        #     continue
        if line_count > 0:
            df = add_track(df, row[13])
            lyrics_onto_frame(df, line_count-1)
            line_count += 1
        else:
            line_count += 1
        if line_count > 17000:
            break
    print(f'Processed {line_count} lines.')

print(len(df))
df = df[df['lyrics'].notnull()]
df = df[df['cover image'].notnull()]
df.to_pickle("final_data.pkl")

100%|██████████████████████████████████████████████████████████████████| 17000/17000 [25:53<00:00, 10.94it/s]

Processed 17001 lines.
17001


In [13]:
print(len(df))
df.loc[0, 'lyrics']

10232
